# Data Expo 2009 - Airline on-time performance
## by (Mahmoud Lotfi)

## Preliminary Wrangling

- The data consists of flight arrival and departure details for all commercial flights within the USA, from October 1987 to April 2008. 
- This is a large dataset: there are nearly 120 million records in total, and takes up 1.6 gigabytes of space compressed and 12 gigabytes when uncompressed. 
- The data comes originally from RITA where it is described in detail. 
- the data in bzipped csv file. 
- These files have derivable variables removed, are packaged in yearly chunks and have been more heavily compressed than the originals.
- in this project we will discuss flight delay for __2008__ data set

### Individual years:

1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008

### Our scope on 2008 dataset

## How to Run the project:
- Insrt raw data download in __"./data/raw"__
- Run the script __"Communicate-Dtata-Finding\src\data\make_dataset.py"__
- Now you can find output data from script  __"Communicate-Dtata-Finding/data/interim/*.csv"__
- Run the notebook in __"Communicate-Dtata-Finding\notebooks\exploration.ipynb"__ to find __Exploratory data__
- Run the notebook in __"Communicate-Dtata-Finding\notebooks\explanatory.ipynb"__ to find __Explanatory data__

In [23]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

# display static image online 
%matplotlib inline

# display zoomable images inline
#%matplotlib notebook

# suppress warnings from final output
import warnings
warnings.simplefilter("ignore")

### User define function and general configrations

In [24]:
'''
# center plot figure
from IPython.core.display import HTML
HTML(
    """
    <style>    .output_png 
    {    
    display: table-cell;    
    text-align: center;    
    vertical-align: 
    middle;
    }
    </style>
    """
    )

'''

'\n# center plot figure\nfrom IPython.core.display import HTML\nHTML(\n    """\n    <style>    .output_png \n    {    \n    display: table-cell;    \n    text-align: center;    \n    vertical-align: \n    middle;\n    }\n    </style>\n    """\n    )\n\n'

In [25]:
# reset seaborn settings
sns.reset_orig()
# set plotting color
base_color = sns.color_palette()[0]

In [26]:
def load_dataset(name='flights'):
    '''
    Description: load dataset acoordding to year
    parameter name string flights, diverted, canceled
    return dataframe
    '''
    t1 = time.time()
    df = pd.read_csv('../data/interim/{}.csv'.format(name))
    t2 = time.time()
    print('Elapsed loading time :', t2-t1)
    return df

In [27]:
def plot_bar_str(df, col, inc=1, base_color=sns.color_palette()[0] , 
                            title = None ,  fontsize =40, 
                            figsize = None, rotation = 0, log =None):
    '''
    Description: custom bar plot
    parameter   df : dataframe
                col: destent colunm 
                inc: increment value for yaxis 
                base_color : bar color
                title: chart title
                fontsize: font size
                figsize: chart size 
                rotation: x-axis label rotation
                log : y-axis log scale
    return dataframe
    '''
    month_frq = df[col].value_counts()
    if figsize: plt.figure(figsize=figsize)
    if not log:
        month_max_count = month_frq[0]
        month_max_prop = month_max_count/1000
        tick_prop = np.arange(0, month_max_prop, inc)
        tick_names = ['{:0.0f}K'.format(v) for v in tick_prop]
        plt.yticks(tick_prop*1000, tick_names)  
        
    ax = plt.gca()
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)
    
    plt.xticks(rotation=rotation)
    
    sns.countplot(data = flights, x=col, color = base_color, ax=ax);
    if  log  : 
        plt.yscale('log')
    if  title: 
        plt.title( label = title, fontsize=fontsize)  
    plt.grid()
    return ax

In [28]:
def sort_time_fmt(df1, col):
    '''
    Description: create new dataframe contain hour, 
                frequency order by hour mintes from string format column dataframe
    parameter :
       df1 : dataframe
       col: target column
    return dataframe
    '''
    df = df1[col].value_counts().to_frame()
    df['hours'] = pd.to_datetime(df.index, format='%I:%M %p').hour
    df['minutes'] = pd.to_datetime(df.index, format='%I:%M %p').minute
    df = df.reset_index()
    df.columns = [col, 'frequency' , 'hours' , 'minutes']
    df = df.sort_values(['hours', 'minutes'])
    df = df.drop(columns=['minutes'])
    df = df.groupby('hours').sum().reset_index()
    df.columns = [col, 'frequency']   
    return df

In [29]:
def plot_line_month(df, x, figsize=(12, 4), title = None ,  fontsize =40, rotation = 0 ):
    '''
    Description: custom line plot
    parameter :
       df : dataframe
       x,y : target column axes
       figsize : chart frame size
       title , fontsize : title label and size
    return dataframe
    '''
    # make it a datetime so that we can sort it: 
    # use %b because the data use the abbriviation of month
    df["Month"] = pd.to_datetime(df.Month, format='%b', errors='coerce').dt.month
    df = df.sort_values(by="Month")
    order = df.index
    max_count = df[y].max()
    max_prop = max_count/1000
    tick_prop = np.arange(0, max_prop, 100)
    tick_names = ['{:0.0f}K'.format(v) for v in tick_prop]
    plt.figure(figsize=figsize)
    ax = plt.gca()
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)
    plt.yticks(tick_prop*1000, tick_names)        
    if  title: plt.title( label=title, fontsize=fontsize)  
    df.plot(x=x, y=y, ax =ax);
    plt.grid()

In [30]:
def plot_line_time_fmt(df, x, y='frequency', figsize=(12, 4),  
                            title = None ,  fontsize =40, 
                            xtitle= None , xfontsize =18,
                            ytitle= None , yfontsize =18, 
                            rotation = 0 ):
    '''
    Description: custom line plot
    parameter :
       df : dataframe
       x,y : target column axes
       figsize : chart frame size
       title , fontsize : title label and size
       xtitle , xfontsize : title label and size
       ytitle , yfontsize : title label and size
       
    return dataframe
    '''
    order = df.index
    max_count = df[y].max()
    max_prop = max_count/1000
    tick_prop = np.arange(0, max_prop, 100)
    tick_names = ['{:0.0f}K'.format(v) for v in tick_prop]
    plt.figure(figsize=figsize)
    ax = plt.gca()
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)
    plt.yticks(tick_prop*1000, tick_names)        
    if  title: plt.title( label=title, fontsize=fontsize)  
    df.plot(x=x, y=y, ax =ax);
    plt.grid()

In [31]:
def plot_hist(col, inc, 
                            title = None ,  fontsize =40, 
                            xtitle= None , xfontsize =18,
                            ytitle= None , yfontsize =18, 
                            figsize = None,rotation = 0,             
                            log =None , scale=1):
    '''
    Description: custom hist plot
    parameter :
       col : col from dataframe 
       inc : y increment value
       figsize : chart frame size
       title , fontsize : title label and size
       xtitle , xfontsize : title label and size
       ytitle , yfontsize : title label and size
       
    return dataframe
    '''
    if scale == 1000:
        k='K'
    else:
        k=''
    inc = 1
    max_prop = (1.25*flights['ActualElapsedTimePmile'].max())/scale
    tick_prop = np.arange(0, max_prop, inc)
    tick_names = ['{:0.0f}{}'.format(v,k) for v in tick_prop]
    if figsize:  plt.figure(figsize=figsize)

    plt.yticks(tick_prop*scale, tick_names)
    ax = plt.gca()

    plt.xticks(rotation=rotation)
    if log: plt.yscale('log')
    plt.grid(alpha= 0.2)
    sns.distplot(flights['ActualElapsedTimePmile'], ax=ax);
    
    plt.xlim(right=1);  # adjust the right leaving left unchanged
    plt.xlim(left=0);  # adjust the left leaving right unchanged
    
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)
    if  title: plt.title( label=title, fontsize=fontsize)  
    if xtitle: plt.xlabel(label=title, fontsize=xfontsize)
    if ytitle: plt.ylabel(label=title, fontsize=yfontsize)
    plt.grid()

In [32]:
'''
# relative frequency
# return the highest frequency 
month_max_count = month_frq[0]
month_max_prop = month_max_count/flights.shape[0]
tick_prop = np.arange(0, month_max_prop, 0.02)
tick_names = ['{:0.2f}'.format(v) for v in tick_prop]
plt.yticks(tick_prop*flights.shape[0], tick_names)
plt.grid()
sns.countplot(data = flights, x='Month', color = base_color, order = month_frq.index);
'''

"\n# relative frequency\n# return the highest frequency \nmonth_max_count = month_frq[0]\nmonth_max_prop = month_max_count/flights.shape[0]\ntick_prop = np.arange(0, month_max_prop, 0.02)\ntick_names = ['{:0.2f}'.format(v) for v in tick_prop]\nplt.yticks(tick_prop*flights.shape[0], tick_names)\nplt.grid()\nsns.countplot(data = flights, x='Month', color = base_color, order = month_frq.index);\n"

#### Run "../src/data/make_dataset.py" to divide the data set into three part for easier handling nan values
the output of the script in:
- ../data/interim/flights.csv
- ../data/interim/diverted.csv
- ../data/interim/canceled.csv

In [ ]:
# now we will load flights dataset 2008
flights = load_dataset()
# print df size
print(flights.shape)
# dispay first 10 rows
flights.head()

In [ ]:
# get df data types and schema
flights.info()

In [ ]:
flights.columns

In [ ]:
totals_flights = flights.shape[0]

In [ ]:
# find the number of duplicated rows
flights.duplicated().sum()

In [ ]:
# drop duplicated rows
flights = flights.drop_duplicates()

In [ ]:
# check number of duplicated rows
flights.duplicated().sum()

In [ ]:
# check for null in each column
# flights.isna().sum()
flights.isnull().sum()

In [ ]:
# drop all rows with any NaN and Null values
flights = flights.dropna()

In [ ]:
# check for null in each column
# flights.isna().sum()
flights.isnull().sum()

In [ ]:
# the size after deleting null
flights.shape[0] - totals_flights

In [ ]:
# change schema
flights['Year']              = flights['Year'].astype('str')
flights['Month']             = flights['Month'].astype('str')
flights['DayofMonth']        = flights['DayofMonth'].astype('str')
flights['DayOfWeek']         = flights['DayOfWeek'].astype('str')

flights['DepTime']           = flights['DepTime'].astype('str')          
flights['CRSDepTime']        = flights['CRSDepTime'].astype('str')        
flights['ArrTime']           = flights['ArrTime'].astype('str')          
flights['CRSArrTime']        = flights['CRSArrTime'].astype('str')        

flights['UniqueCarrier']     = flights['UniqueCarrier'].astype('str')  
flights['FlightNum']         = flights['FlightNum'].astype('str')  
flights['TailNum']           = flights['TailNum'].astype('str') 

flights['ActualElapsedTime'] = flights['ActualElapsedTime'].astype('int') 
flights['CRSElapsedTime']    = flights['CRSElapsedTime'].astype('int') 
flights['AirTime']           = flights['AirTime'].astype('int') 
flights['ArrDelay']          = flights['ArrDelay'].astype('int') 
flights['DepDelay']          = flights['DepDelay'].astype('int') 

flights['Origin']            = flights['Origin'].astype('str') 
flights['Dest']              = flights['Dest'].astype('str') 

flights['Distance']          = flights['Distance'].astype('int') 

flights['TaxiIn']            = flights['TaxiIn'].astype('int')  
flights['TaxiOut']           = flights['TaxiOut'].astype('int') 

flights['CarrierDelay']      = flights['CarrierDelay'].astype('int')  
flights['WeatherDelay']      = flights['WeatherDelay'].astype('int') 
flights['NASDelay']          = flights['NASDelay'].astype('int') 
flights['SecurityDelay']     = flights['SecurityDelay'].astype('int') 
flights['LateAircraftDelay'] = flights['LateAircraftDelay'].astype('int') 

In [ ]:
# get df data types and schema
flights.info()

In [ ]:
flights.describe().astype(int)

## 01. Flights without cancellation nor divertion dataset:

### What is the structure of your dataset?

> There are 6,851,832 flight observations with 26 features in 2008 without diverted/cancelled flights and missing or incorrect​ data.


### What is/are the main feature(s) of interest in your dataset?

> Delayed flights in terms of carriers, origin & time.


### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

> ArrDelay, Month, DayOfWeek, DepTime, ArrTime, UniqueCarrier.

In [ ]:
# now we will load flights dataset 2008
cancelled = load_dataset('canceled')
# print df size
print(cancelled.shape)
# dispay first 10 rows
cancelled.head()

In [ ]:
# get df data types and schema
cancelled.info()

In [ ]:
cancelled.columns

In [ ]:
totals_cancelled = cancelled.shape[0]

In [ ]:
# find the number of duplicated rows
cancelled.duplicated().sum()

In [ ]:
# check for null in each column
# flights.isna().sum()
cancelled.isnull().sum()

In [ ]:
# drop all columns with any NaN and NaT values
cancelled = cancelled.dropna(axis=1)

In [ ]:
# check for null in each column
# flights.isna().sum()
cancelled.isnull().sum()

In [ ]:
# the size after deleting null
cancelled.shape[0] - totals_cancelled

In [ ]:
# change schema
cancelled['Year']              = cancelled['Year'].astype('str')
cancelled['Month']             = cancelled['Month'].astype('str')
cancelled['DayofMonth']        = cancelled['DayofMonth'].astype('str')
cancelled['DayOfWeek']         = cancelled['DayOfWeek'].astype('str')

cancelled['CRSDepTime']        = cancelled['CRSDepTime'].astype('str')        
cancelled['CRSArrTime']        = cancelled['CRSArrTime'].astype('str')        

cancelled['UniqueCarrier']     = cancelled['UniqueCarrier'].astype('str')  
cancelled['FlightNum']         = cancelled['FlightNum'].astype('str')  

cancelled['Origin']            = cancelled['Origin'].astype('str') 
cancelled['Dest']              = cancelled['Dest'].astype('str') 

cancelled['Distance']          = cancelled['Distance'].astype('int') 
cancelled['CancellationCode']  = cancelled['CancellationCode'].astype('str') 

In [ ]:
# get df data types and schema
cancelled.info()

In [ ]:
cancelled.describe().astype(int)

## 02. Flights that have been Cancellation:

### What is the structure of your dataset?

> There are 137,434 flight observations with 12 features in 2008.


### What is/are the main feature(s) of interest in your dataset?

 - what are the worstest airlines in terms of cancelled flighes?
 - what are the most cases of flights cancelled?


### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

> UniqueCarrier, CancellationCode

In [ ]:
# now we will load flights dataset 2008
diverted = load_dataset('diverted')
# print df size
print(diverted.shape)
# dispay first 10 rows
diverted.head()

In [ ]:
# get df data types and schema
diverted.info()

In [ ]:
diverted.columns

In [ ]:
totals_diverted = diverted.shape[0]

In [ ]:
# find the number of duplicated rows
diverted.duplicated().sum()

In [ ]:
# check for null in each column
# flights.isna().sum()
diverted.isnull().sum()

In [ ]:
# drop all columns with any NaN and NaT values
diverted = diverted.dropna(axis=1)

In [ ]:
# check for null in each column
# flights.isna().sum()
diverted.isnull().sum()

In [ ]:
# the size after deleting null
diverted.shape[0] - totals_diverted

In [ ]:
# change schema
diverted['Year']              = diverted['Year'].astype('str')
diverted['Month']             = diverted['Month'].astype('str')
diverted['DayofMonth']        = diverted['DayofMonth'].astype('str')
diverted['DayOfWeek']         = diverted['DayOfWeek'].astype('str')

diverted['DepTime']           = diverted['DepTime'].astype('str')          
diverted['CRSDepTime']        = diverted['CRSDepTime'].astype('str')        

diverted['UniqueCarrier']     = diverted['UniqueCarrier'].astype('str')  
diverted['FlightNum']         = diverted['FlightNum'].astype('str')  

diverted['DepDelay']          = diverted['DepDelay'].astype('int') 

diverted['Origin']            = diverted['Origin'].astype('str') 
diverted['Dest']              = diverted['Dest'].astype('str') 

diverted['Distance']          = diverted['Distance'].astype('int') 

In [ ]:
# get df data types and schema
diverted.info()

In [ ]:
diverted.describe().astype(int)

## 03. Flights that have been diverted:

### What is the structure of your dataset?

> There are 17,265 flight observations with 12 features in 2008.


### What is/are the main feature(s) of interest in your dataset?

> what are the Origin & Dest that have the most diverted flighes?


### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

> Origin & Dest.

## Expoloring Flights dataset:

### 01. Flight dataset Year column:
    range :	 1987-2008 

In [ ]:
flights.Year.value_counts()

### 02. Flight dataset Month column:
    range :	 1-12                                                        

In [ ]:
flights.Month.describe()

In [ ]:
months = ['', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 
           'August', 'September', 'October', 'November', 'December']
# substitute number with actual day of month name 
for i in flights.Month.unique():
    if str(i).isnumeric():
        flights.Month.replace(i,months[int(i)], inplace=True)

In [ ]:
flights.Month.unique()

In [ ]:
plot_bar_str(flights, 'Month', 100, base_color, figsize=(11, 5), title = 'The Number Of Flights Per Month', fontsize =14); 
plt.xlabel('Months In 2008',fontsize =14);                          
plt.ylabel('Number Of Flights',fontsize =14);

### Bar plot indicate the number of flights per month

In [ ]:
plot_bar_str(flights, 'Month', 100, base_color, figsize=(11, 5), title = 'The Number Of Flights Per Month', fontsize =14, log =True)
plt.xlabel('Months In 2008',fontsize =14);                          
plt.ylabel('Number Of Flights',fontsize =14);

- less flights on winter November , December  

### 03. Flight dataset DayofMonth column:
    range :	 1-31

In [ ]:
flights.DayofMonth.describe()

### Bar plot indicate the number of flights per day of month

In [ ]:
plot_bar_str(flights, 'DayofMonth', 50, base_color, figsize=(11, 5), title = 'The Number Of Flights Per Day of Month', fontsize =14);
plt.xlabel('Days of Months In 2008',fontsize =12);                          
plt.ylabel('Number Of Flights',fontsize =12);

- There are not big difference amonge monthes in the flights number during the days of months, and its natural to be less half flights during 31th.

### 04. Flight dataset DayOfWeek column:
    range :	 1 (Monday) - 7 (Sunday)

In [ ]:
# substitute number with actual day of week name 
days = ['', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
for i in flights.DayOfWeek.unique():
    if str(i).isnumeric():
        flights.DayOfWeek.replace(i,days[int(i)], inplace=True)

In [ ]:
flights.DayOfWeek.describe()

In [ ]:
plot_bar_str(flights, 'DayOfWeek', 200, base_color, figsize=(11, 5), title = 'The Number Of Flights Per Day', fontsize =14);
plt.xlabel('Days',fontsize =12);                          
plt.ylabel('Number Of Flights',fontsize =12);

#### Bar plot indicate the number of flights per day of week

In [ ]:
plot_bar_str(flights, 'DayOfWeek', 200, base_color, rotation =45, log=True)

There are not big difference amonge flights number during working days 'Tuesday', 'Wednesday', 'Thursday', 'Friday' , and less  flights during Sunday , and more less in Saturday.

### 05. Flight dataset DepTime column:
    range :	 00:00 AM-11:59 PM  
    actual departure time (local, hh:mm AM/PM)

In [ ]:
flights.DepTime.describe()

In [ ]:
# sort time in ascending order with its frequency
df = sort_time_fmt(flights, 'DepTime')

In [ ]:
df[df.frequency == df.frequency.max()]

### line plot indicate the number of actual departed flights over the day time

In [ ]:
plot_line_time_fmt(df, 'DepTime', 'frequency', figsize=(12, 4))

### 06. Flight dataset CRSDepTime column:
    range :	 00:00 AM-11:59 PM  
    scheduled departure time (local, hh:mm AM/PM)

In [ ]:
flights.CRSDepTime.describe()

In [ ]:
df = sort_time_fmt(flights, 'CRSDepTime')
df.head()

In [ ]:
df[df.frequency == df.frequency.max()]

### line plot indicate the number of scheduled departed flights over the day time

In [ ]:
plot_line_time_fmt(df, 'CRSDepTime', 'frequency', figsize=(12, 4))

### 07. Flight dataset ArrTime column:
    range :	 00:00 AM-11:59 PM  
    actual arrival time (local, hh:mm AM/PM)

In [ ]:
flights.ArrTime.describe()

In [ ]:
df = sort_time_fmt(flights, 'ArrTime')
df.head()

In [ ]:
df[df.frequency == df.frequency.max()]

### line plot indicate the number of actual arrival flights over the day time

In [ ]:
plot_line_time_fmt(df, 'ArrTime', 'frequency', figsize=(12, 4))

### 08. Flight dataset CRSArrTime column:
    range :	 00:00 AM-11:59 PM  
    scheduled arrival time (local, hh:mm AM/PM)

In [ ]:
flights.CRSArrTime.describe()

In [ ]:
df = sort_time_fmt(flights, 'CRSArrTime')
df.head()

In [ ]:
df[df.frequency == df.frequency.max()]

### line plot indicate the number of scheduled arrival flights over the day time

In [ ]:
plot_line_time_fmt(df, 'CRSArrTime', 'frequency', figsize=(12, 4))

### 09. Flight dataset UniqueCarrier column:
    range :	 1 - 20 string
    unique carrier code 

In [ ]:
flights.UniqueCarrier.describe()

#### Bar plot indicate the number of flights per carrier

In [ ]:
plot_bar_str(flights, 'UniqueCarrier', 200, base_color, (10, 4), rotation =0)

apperantly WN carrier have the most share of flights about 1,186,911 flight.

In [ ]:
plot_bar_str(flights, 'UniqueCarrier', 200, base_color, (10, 4), rotation =0, log=True)

### 10. Flight dataset FlightNum column:
    range :	string
    flight number 

In [ ]:
flights.FlightNum.describe()

### 11. Flight dataset TailNum column:
    range :	string
    plane tail number :
    aircraft registration, unique aircraft identifier

In [ ]:
flights.TailNum.describe()

# keep the orignal then remove outliers

In [ ]:
flights_orignal = flights.copy()

In [ ]:
#flights = flights_orignal

outliers = (((df.DepDelay - df.DepDelay.mean()).abs() > df.DepDelay.std()*3) | 
            ((df.ArrDelay - df.ArrDelay.mean()).abs() > df.ArrDelay.std()*3))

In [ ]:
def outlier(df, col):
    df_non_outlier = df[(df[col] >= (2.5*df[col].quantile(.25)-1.5*df[col].quantile(.75))) & (df[col] <= (2.5*df[col].quantile(.75)-1.5*df[col].quantile(.25)))]
    return df_non_outlier

### 12. Flight dataset ActualElapsedTime column:
    range :	in minutes
    ActualElapsedTime 

In [ ]:
flights.ActualElapsedTime.describe().round(2)

In [ ]:
# remove outliers 
flights = outlier(flights, 'ActualElapsedTime')

In [ ]:
flights.ActualElapsedTime.describe().round(2)

In [ ]:
flights['ActualElapsedTimePmile'] = flights['ActualElapsedTime'].astype(float)/ flights['Distance'].astype(float)

In [ ]:
flights.ActualElapsedTimePmile.describe().round(3)

In [ ]:
bins = np.arange(0,flights['ActualElapsedTimePmile'].max()+0.1,0.1)
plt.hist(data = flights, x= 'ActualElapsedTimePmile', bins=bins);
plt.xlim(right=1);  # adjust the right leaving left unchanged
plt.xlim(left=0);  # adjust the left leaving right unchanged

In [ ]:
bins = np.arange(0,flights['ActualElapsedTimePmile'].max()+0.1,0.1)
plt.hist(data = flights, x= 'ActualElapsedTimePmile', bins=bins);
plt.xlim(right=4);  # adjust the right leaving left unchanged
plt.xlim(left=-0.5);  # adjust the left leaving right unchanged
plt.yscale('log')

#### histogram indicate the density of Actual ElapsedTime for flights per distance

In [ ]:
plot_hist(flights['ActualElapsedTimePmile'], 1, scale=1, x_right=1, x_left=0)

In [ ]:
sns.boxplot(x=flights['ActualElapsedTimePmile']);
plt.xlim(right=0.4);  # adjust the right leaving left unchanged
plt.xlim(left=0);  # adjust the left leaving right unchanged

In [ ]:
flights.boxplot(column=['ActualElapsedTimePmile']);
plt.ylim(top=0.4);  # adjust the top leaving bottom unchanged
plt.ylim(bottom=0); # adjust the bottom leaving top unchanged

### 13. Flight dataset CRSElapsedTime column:
    range :	in minutes
    CRSElapsedTime

In [ ]:
flights.CRSElapsedTime.describe().round(1)

In [ ]:
flights[flights['CRSElapsedTime'] <0]

#### remove incorrect data

In [ ]:
flights = flights[~(flights['CRSElapsedTime'] <0)]

In [ ]:
flights.CRSElapsedTime.describe().round(1)

In [ ]:
# remove outliers 
flights = outlier(flights, 'CRSElapsedTime')

In [ ]:
flights.CRSElapsedTime.describe().round(1)

In [ ]:
flights['CRSElapsedTimePmile'] = flights['CRSElapsedTime'].astype(float)/ flights['Distance'].astype(float)

In [ ]:
flights.ActualElapsedTimePmile.describe().round(3)

In [ ]:
bins = np.arange(0,flights['CRSElapsedTimePmile'].max()+0.1,0.1)
plt.hist(data = flights, x= 'CRSElapsedTimePmile', bins=bins);
plt.xlim(right=1);  # adjust the right leaving left unchanged
plt.xlim(left=0);  # adjust the left leaving right unchanged

#### histogram indicate the density of scheduled ElapsedTime for flights per distance

In [ ]:
plot_hist(flights['CRSElapsedTimePmile'], 1, scale=1, x_right=1, x_left=0)

In [ ]:
sns.distplot(flights['CRSElapsedTimePmile']);
plt.xlim(right=1);  # adjust the right leaving left unchanged
plt.xlim(left=0);  # adjust the left leaving right unchanged

In [ ]:
sns.boxplot(x=flights['CRSElapsedTimePmile']);
plt.xlim(right=0.5);  # adjust the right leaving left unchanged
plt.xlim(left=0);  # adjust the left leaving right unchanged

In [ ]:
flights.boxplot(column=['CRSElapsedTimePmile']);
plt.ylim(top=0.5);  # adjust the top leaving bottom unchanged
plt.ylim(bottom=0); # adjust the bottom leaving top unchanged

### 14. Flight dataset AirTime column:
    range :	in minutes

In [ ]:
flights.AirTime.describe().round(1)

In [ ]:
# remove outliers 
flights = outlier(flights, 'AirTime')

In [ ]:
flights.AirTime.describe().round(1)

In [ ]:
flights['AirTimePmile'] = flights['AirTime'].astype(float)/ flights['Distance'].astype(float)

In [ ]:
flights.AirTimePmile.describe().round(3)

In [ ]:
bins = np.arange(0,flights['AirTimePmile'].max()+0.1,0.1)
plt.hist(data = flights, x= 'AirTimePmile', bins=bins);
plt.xlim(right=1);  # adjust the right leaving left unchanged
plt.xlim(left=0);  # adjust the left leaving right unchanged

#### histogram indicate the density of arive Time for flights per distance

In [ ]:
plot_hist(flights['AirTimePmile'], 1, scale=1, x_right=1, x_left=0)

In [ ]:
sns.distplot(flights['AirTimePmile']);
plt.xlim(right=1);  # adjust the right leaving left unchanged
plt.xlim(left=0)  # adjust the left leaving right unchanged

In [ ]:
sns.boxplot(x=flights['AirTimePmile']);
plt.xlim(right=0.35);  # adjust the right leaving left unchanged
plt.xlim(left=0);  # adjust the left leaving right unchanged

In [ ]:
flights.boxplot(column=['AirTimePmile']);
plt.ylim(top=0.35);  # adjust the top leaving bottom unchanged
plt.ylim(bottom=0); # adjust the bottom leaving top unchanged

### 15. Flight dataset ArrDelay column:
    range :	in minutes
    arrival delay:
    A flight is counted as "on time" if it operated less than 15 minutes later the scheduled time shown in the carriers' Computerized Reservations Systems (CRS).

In [ ]:
flights.ArrDelay.describe().round(1)

In [ ]:
flights[flights['ArrDelay'] <15].shape

In [ ]:
# remove outliers 
flights = outlier(flights, 'ArrDelay')

In [ ]:
flights.ArrDelay.describe().round(1)

In [ ]:
plt.hist(data = flights, x= 'ArrDelay');
#plt.xlim(right=500);  # adjust the right leaving left unchanged
plt.xlim(left=-60);  # adjust the left leaving right unchanged

In [ ]:
plt.hist(data = flights, x= 'ArrDelay');
#plt.xlim(right=500);  # adjust the right leaving left unchanged
plt.xlim(left=-60);  # adjust the left leaving right unchanged
plt.yscale('log')

In [ ]:
plot_hist(flights['AirTimePmile'], .1)

#### histogram indicate the density of arive delayTime for flights per distance

In [ ]:
sns.distplot(flights['ArrDelay']);
#plt.xlim(right=12);  # adjust the right leaving left unchanged
#plt.xlim(left=-700);  # adjust the left leaving right unchanged
#plt.yscale('log')

In [ ]:
sns.boxplot(x=flights['ArrDelay']);
plt.xlim(right=50);  # adjust the right leaving left unchanged
plt.xlim(left=-50);  # adjust the left leaving right unchanged

In [ ]:
flights.boxplot(column=['ArrDelay']);
plt.ylim(top=50);  # adjust the top leaving bottom unchanged
plt.ylim(bottom=-50); # adjust the bottom leaving top unchanged

### 16. Flight dataset DepDelay column:
    range :	in minutes
    departure delay:

In [ ]:
flights.DepDelay.describe().round(1)

In [ ]:
flights[flights['DepDelay'] <0].shape

In [ ]:
# remove outliers 
flights = outlier(flights, 'DepDelay')

In [ ]:
flights.DepDelay.describe().round(1)

In [ ]:
plt.hist(data = flights, x= 'DepDelay');
#plt.xlim(right=1);  # adjust the right leaving left unchanged
#plt.xlim(left=0);  # adjust the left leaving right unchanged

#### histogram indicate the density of depate delay Time for flights per distance

In [ ]:
plot_hist(flights['AirTimePmile'], .1)

In [ ]:
sns.distplot(flights['DepDelay']);
#plt.xlim(right=1);  # adjust the right leaving left unchanged
#plt.xlim(left=0)  # adjust the left leaving right unchanged

In [ ]:
sns.boxplot(x=flights['DepDelay']);
#plt.xlim(right=1);  # adjust the right leaving left unchanged
#plt.xlim(left=0);  # adjust the left leaving right unchanged

In [ ]:
flights.boxplot(column=['DepDelay']);
#plt.ylim(top=1);  # adjust the top leaving bottom unchanged
#plt.ylim(bottom=0); # adjust the bottom leaving top unchanged

### 17. Flight dataset Origin column:
    range :	string
    origin IATA airport code

In [ ]:
flights.Origin.describe()

### 18. Flight dataset Dest column:
    range :	string
    destination IATA airport code 

In [ ]:
flights.Dest.describe()

### 19. Flight dataset Distance column:
    range :	in miles 
    Distance

In [ ]:
flights.Distance.describe().round(1)

### 20. Flight dataset TaxiIn column:
    range :	in minutes
    taxi in time 

In [ ]:
flights.TaxiIn.describe().round(1)

In [ ]:
# remove outliers 
flights = outlier(flights, 'TaxiIn')

In [ ]:
flights.TaxiIn.describe().round(1)

### 21. Flight dataset TaxiOut column:
    range :	in minutes 
    taxi out time 

In [ ]:
flights.TaxiOut.describe().round(1)

In [ ]:
# remove outliers 
flights = outlier(flights, 'TaxiOut')

In [ ]:
flights.TaxiOut.describe().round(1)

### 25. Flight dataset CarrierDelay column:
    range :	in minutes
    - Carrier delay is within the control of the air carrier. 
    - Examples of occurrences that may determine carrier delay are: 
        - aircraft cleaning, aircraft damage, awaiting the arrival of connecting passengers or crew, 
        - baggage, bird strike, cargo loading, catering, computer, outage-carrier equipment, 
        - crew legality (pilot or attendant rest), damage by hazardous goods, engineering inspection, fueling, 
        - handling disabled passengers, late crew, lavatory servicing, maintenance, oversales, 
        - potable water servicing, removal of unruly passenger, slow boarding or seating, 
        - stowing carry-on baggage, weight and balance delays.              

In [ ]:
flights.CarrierDelay.describe().round(3)

### 26. Flight dataset WeatherDelay column:
    range :	in minutes
    Weather delay is caused by extreme or hazardous weather conditions that are forecasted or 
    manifest themselves on point of departure, enroute, or on point of arrival. 


In [ ]:
flights.WeatherDelay.describe().round(3)

### 27. Flight dataset NASDelay column:
    range :	in minutes
    Delay that is within the control of the National Airspace System (NAS) may include: 
    non-extreme weather conditions, airport operations, heavy traffic volume, air traffic control, etc.          

In [ ]:
flights.NASDelay.describe().round(3)

### 28. Flight dataset SecurityDelay column:
    range :	in minutes
    Security delay is caused by evacuation of a terminal or concourse, re-boarding of aircraft because of security breach, inoperative screening equipment and/or long lines in excess of 29 minutes at screening areas.

In [ ]:
flights.SecurityDelay.describe().round(3)

### 29. Flight dataset LateAircraftDelay column:
    range :	in minutes
    Arrival delay at an airport due to the late arrival of the same aircraft at a previous airport. 
    The ripple effect of an earlier delay at downstream airports is referred to as delay propagation. 

In [ ]:
flights.LateAircraftDelay.describe().round(3)

## correlation matrix

In [ ]:
corrmat = flights.corr()
f, ax = plt.subplots(figsize=(15, 10))
sns.heatmap(corrmat, vmin=-1, square=True, annot=True, fmt='.2f', cmap='vlag_r', center=0);

## Saving cleaned data after romvoing outliers and duplicates

In [ ]:
flights.columns

In [ ]:
# drop some columns
flights = flights[['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay',
       'LateAircraftDelay']]

In [ ]:
# saving data
t1 = time.time()
flights.to_csv('../data/processed/{}.csv'.format('flights'), index=False)
t2 = time.time()
print('Elapsed saving time :', t2-t1)

In [ ]:
cancelled.columns

In [ ]:
# drop some columns
cancelled = cancelled[['Month', 'DayofMonth', 'DayOfWeek', 'CRSDepTime', 'CRSArrTime',
       'UniqueCarrier', 'FlightNum', 'Origin', 'Dest', 'Distance', 'CancellationCode']]

In [ ]:
# saving data
t1 = time.time()
cancelled.to_csv('../data/processed/{}.csv'.format('cancelled'), index=False)
t2 = time.time()
print('Elapsed saving time :', t2-t1)

In [ ]:
diverted.columns

In [ ]:
# drop some columns
diverted = diverted[['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'UniqueCarrier', 'FlightNum', 'DepDelay', 'Origin', 'Dest', 'Distance']]

In [ ]:
# saving data
t1 = time.time()
diverted.to_csv('../data/processed/{}.csv'.format('diverted'), index=False)
t2 = time.time()
print('Elapsed saving time :', t2-t1)

In [ ]:
!jupyter nbconvert exploration.ipynb --to slides --post serve --no-input --no-prompt

> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report!